In [144]:
import pandas as pd
import re
from collections import Counter
import networkx as nx
import math
from ipysigma import Sigma
import fileinput

In [145]:
df = pd.read_csv("trademarks.csv")

In [146]:
trademarks = df.trademark.to_list()

In [147]:
trademarks += ["iPhone Pro", "iPhone Pro Max", "iPhone mini", "iPhone Plus", "iPhone SE", "iOS"]

In [148]:
cleaned = dict()
fragments = Counter()
for t in trademarks:
    bad = False
    for ex in ["logo", "works", "@", "symbol"]:
        bad |= ex in t.lower()
    if not bad:
        t = t.replace("+", "Plus")
        t = t.replace("-", "")
        result = []
        for x in re.sub( r"([A-Z])", r" \1", t).split():
            if len(result) > 0 and len(x) == 1 and result[-1][-1].upper() == result[-1][-1]:
                result[-1] += x
            else:
                result.append(x)
        if result[0] == "OSX":
            result = ["OS", "X"]
        if result[0][0] == "X":
            result = ["X", result[0][1:]]
        result = [r.lower() for r in result]
        plurals = ["pods", "clips", "photos", "messages", "books"]
        result = [r if r not in plurals else r[:-1] for r in result]
        if len(result) > 1:
            cleaned[t] = result
            fragments.update(result)

In [149]:
cleaned2 = dict()
fragments2 = Counter()
for name, words in cleaned.items():
    ok = False
    for w in words:
        ok |= fragments[w] > 1
    if ok:
        cleaned2[name] = words
        fragments2.update(words)

In [150]:
frag2id = {w:i for i, w in enumerate(fragments2.keys())}

In [151]:
G = nx.DiGraph()
for w, i in frag2id.items():
    G.add_node(i, label=w, size=fragments2[w])

weighted_edges = Counter()
for name, words in cleaned2.items():
    prev = None
    for w in words:
        if prev:
            weighted_edges.update([(prev,w)])
        prev = w

for (source, target), count in weighted_edges.items():
    G.add_edge(frag2id[source], frag2id[target])

In [153]:
Sigma.write_html(G, path="index.html", node_size=G.degree, node_color=G.out_degree, start_layout=5, hide_info_panel=True, 
      hide_search=True, default_edge_size=1.5, node_color_gradient='Plasma', fullscreen=True)


with fileinput.FileInput("index.html", inplace=True, backup='.bak') as file:
    for line in file:
        print(
            line
            .replace("<body>", "<style>.ipysigma-left-panel, .ipysigma-right-panel {display: none;}</style>"+
                     '''<div style="position: absolute; left: 25px; top: 5px; z-index: 100; font-family: Arial;">
                     <h2>How Apple Names Things</h2>
                     <p>by <a href="https://nicolas.kruchten.com/">Nicolas Kruchten</a> using <a href="https://www.apple.com/legal/intellectual-property/trademark/appletmlist.html">Apple trademark data</a>
                     </p>
                     </div>'''+
                     "<body>")
            .replace("IPyWidget export", "How Apple Names Things")
            , end='')
